In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### Herding Analysis

In [3]:
def herding_bondMarket(year_start, year_end):
    
    base_query = '''
         SELECT
            Date,
            Rm,
            ABS(Rm) AS AbsoluteRm,
            POWER(Rm, 2) AS SquaredRm,
            Sum / Count AS Csad,
            CASE 
                WHEN Rm <= [dbo].[leftTailBondReturns]() THEN 1
                ELSE 0
            END AS LeftTail,
            CASE 
                WHEN Rm >= [dbo].[rightTailBondReturns]() THEN 1
                ELSE 0
            END AS RightTail
        FROM (
            SELECT
                Date,
                Rm,
                ABS(SUM(RetEom) - Rm) AS Sum,
                COUNT(DISTINCT Cusip) AS Count
            FROM (
                SELECT
                    A.Date,
                    A.Cusip,
                    A.RetEom,
                    B.Rm
                FROM
                    BondReturns A
                INNER JOIN (
                    -- MONTHLY RM
                    SELECT
                        Date,
                        SUM(RetEom / 100 * TDvolume) / SUM(TDvolume) AS Rm
                    FROM
                        BondReturns
                    WHERE
                        Date >= '{}-01-01' AND Date < '{}-01-01'
                    GROUP BY
                        Date
                ) B ON A.Date = B.Date

            ) A
            GROUP BY
                Date,
                Rm
        ) B
        ORDER BY
            Date
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [4]:
start, end = 2002, 2022

df = herding_bondMarket(year_start=start, year_end=end)
df.to_csv('data/output/bonds/herding_bondMarket_{}-{}.csv'.format(start, end), index=False)
df.head()

,Date,Rm,AbsoluteRm,SquaredRm,Csad,LeftTail,RightTail
0,2002-07-31,-0.002184,0.002184,0.000005,0.072513,0,0
1,2002-08-31,0.034135,0.034135,0.001165,1.890931,0,1
2,2002-09-30,0.003112,0.003112,0.000010,0.987137,0,0
3,2002-10-31,-0.007160,0.007160,0.000051,0.820465,0,0
4,2002-11-30,0.043848,0.043848,0.001923,3.016087,0,1
